In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from holidays.countries import US
import numpy as np

### DEFINE

In [2]:
# 讀取檔案位置

# 初始年月份
set_data_year = 2019 #start_year
set_data_month = 2 #start_month
end_year = 2019
end_month = 12

# 檔案位置
Site = './data/'

# 檔案輸出位置
custom_output_path_dfTime = './dfTime/test'
custom_output_path_dfTLC = './dfTLC/test'

# 年月
start_year = set_data_year
start_month = set_data_month

datasite_fhvhv = []
datasite_yellow = []
for year in range(start_year, end_year+1):  # 修改为你的年份范围
    if year == start_year:
        if year < end_year :
            for month in range(start_month, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)                
        elif year == end_year:
            for month in range(start_month, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)    
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
    elif year > start_year:
        if year < end_year :
            for month in range(1, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
        elif year == end_year:
            for month in range(1, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)         
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   



print("done時間:", datetime.datetime.now())

done時間: 2023-08-17 16:33:37.946931


### 留下需要特徵 & 移除不在當月資料
    HVFHV: 'hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','base_passenger_fare'
    
    Yellow: 'tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID','fare_amount'

In [3]:
# 記錄開始時間
start_time = time.time()


# 建立一個空的DataFrame來存放合併後的數據
df_fhvhv = pd.DataFrame()
df_yellow = pd.DataFrame()

data_year = set_data_year
data_month = set_data_month
temp0_total_fhvfv = 0
temp1_total_fhvfv = 0
temp0_total_yellow = 0
temp1_total_yellow = 0
# 逐一讀取並合併所有Parquet檔案
print('fhvfv==初始筆數=剩餘筆數=移除佔比')
for file in datasite_fhvhv:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    temp_0 = df.shape[0]
    temp0_total_fhvfv += temp_0
    # 留下需要的欄位
    df =  df[['hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','shared_match_flag']]
    # 移除非當月的data
    df =  df[(df['pickup_datetime'].dt.year == data_year) & (df['pickup_datetime'].dt.month == data_month)]
    temp_1 = df.shape[0]
    temp1_total_fhvfv += temp_1
    # 查看剩餘資料量
    if temp_0 > temp_1 :
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,round(100-(temp_1/temp_0*100),5),"%")
    else:
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=0%')
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_fhvhv = pd.concat([df_fhvhv, df], ignore_index=True)
    
    

data_year = set_data_year
data_month = set_data_month
print('yellow==初始筆數=剩餘筆數=移除佔比')
for file in datasite_yellow:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    temp_0 = df.shape[0]
    temp0_total_yellow += temp_0
    # 留下需要的欄位
    df =  df[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
    # 移除非當月的data
    df =  df[(df['tpep_pickup_datetime'].dt.year == data_year) & (df['tpep_pickup_datetime'].dt.month == data_month)]
    temp_1 = df.shape[0]
    temp1_total_yellow += temp_1
    # 查看剩餘資料量
    if temp_0 > temp_1 :
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=',round(100-(temp_1/temp_0*100),5),"%")
    else:
        print(data_year,'=',data_month,'=',temp_0,'=',temp_1,'=0%')        
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_yellow = pd.concat([df_yellow, df], ignore_index=True)
    
    
print(f'fhvfv={start_year}=初始總筆數=剩餘總筆數=移除佔比')
if temp0_total_fhvfv > temp1_total_fhvfv :
    print('\==',temp0_total_fhvfv,'=',temp1_total_fhvfv,'=',round(100-(temp1_total_fhvfv/temp0_total_fhvfv*100),5),"%")
else:
    print('\==',temp0_total_fhvfv,'=',temp1_total_fhvfv,'=0%')

print(f'yellow={start_year}=初始總筆數=剩餘總筆數=移除佔比')
if temp0_total_yellow > temp1_total_yellow :
    print('\==',temp0_total_yellow,'=',temp1_total_yellow,'=',round(100-(temp1_total_yellow/temp0_total_yellow*100),5),"%")
else:
    print('\==',temp0_total_yellow,'=',temp1_total_yellow,'=0%')

# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")


#清理暫存
del data_month,data_year,df,temp_0,temp_1,temp0_total_yellow,temp1_total_yellow
print("done時間:", datetime.datetime.now())

fhvfv==初始筆數=剩餘筆數=移除佔比
2019 = 2 = 20159102 = 20159102 =0%
2019 = 3 = 23864598 = 23864598 =0%
2019 = 4 = 21734822 = 21734822 =0%
2019 = 5 = 22329247 = 22329247 =0%
2019 = 6 = 21001990 = 21001990 =0%
2019 = 7 = 20303312 = 20303312 =0%
2019 = 8 = 20126113 = 20126113 =0%
2019 = 9 = 20069321 = 20069321 =0%
2019 = 10 = 21162290 = 21162290 =0%
2019 = 11 = 21635568 = 21635568 =0%
2019 = 12 = 22243901 = 22243901 =0%
yellow==初始筆數=剩餘筆數=移除佔比
2019 = 2 = 7049370 = 7048744 = 0.00888 %
2019 = 3 = 7866620 = 7866110 = 0.00648 %
2019 = 4 = 7475949 = 7475636 = 0.00419 %
2019 = 5 = 7598445 = 7598068 = 0.00496 %
2019 = 6 = 6971560 = 6971026 = 0.00766 %
2019 = 7 = 6310419 = 6310134 = 0.00452 %
2019 = 8 = 6073357 = 6072851 = 0.00833 %
2019 = 9 = 6567788 = 6567396 = 0.00597 %
2019 = 10 = 7213891 = 7213588 = 0.0042 %
2019 = 11 = 6878111 = 6877463 = 0.00942 %
2019 = 12 = 6896317 = 6895933 = 0.00557 %
fhvfv=2019=初始總筆數=剩餘總筆數=移除佔比
\== 234630264 = 234630264 =0%
yellow=2019=初始總筆數=剩餘總筆數=移除佔比
\== 76901827 = 76896949 = 0

In [4]:
df_fhvhv

,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,shared_match_flag
0,HV0003,2019-02-01 00:05:18,2019-02-01 00:14:57,245,251,2.450,579,N
1,HV0003,2019-02-01 00:41:29,2019-02-01 00:49:39,216,197,1.710,490,N
2,HV0005,2019-02-01 00:51:34,2019-02-01 01:28:29,261,234,5.010,2159,Y
3,HV0005,2019-02-01 00:03:51,2019-02-01 00:07:16,87,87,0.340,179,Y
4,HV0005,2019-02-01 00:09:44,2019-02-01 00:39:56,87,198,6.840,1799,Y
...,...,...,...,...,...,...,...,...
234630259,HV0005,2019-12-31 23:46:50,2020-01-01 00:00:01,10,197,0.000,791,N
234630260,HV0005,2019-12-31 23:30:42,2019-12-31 23:42:59,89,72,2.049,737,N
234630261,HV0005,2019-12-31 23:53:58,2020-01-01 00:15:51,35,89,3.434,1313,N
234630262,HV0003,2019-12-31 23:24:30,2019-12-31 23:31:01,211,113,1.220,391,N


In [5]:
df_yellow

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID
0,2019-02-01 00:59:04,2019-02-01 01:07:27,2.10,48,234
1,2019-02-01 00:33:09,2019-02-01 01:03:58,9.80,230,93
2,2019-02-01 00:09:03,2019-02-01 00:09:16,0.00,145,145
3,2019-02-01 00:45:38,2019-02-01 00:51:10,0.80,95,95
4,2019-02-01 00:25:30,2019-02-01 00:28:14,0.80,140,263
...,...,...,...,...,...
76896944,2019-12-31 23:56:29,2020-01-01 00:11:17,2.82,143,141
76896945,2019-12-31 23:11:53,2019-12-31 23:30:56,3.75,148,246
76896946,2019-12-31 23:57:21,2020-01-01 00:23:34,6.46,197,205
76896947,2019-12-31 23:37:29,2020-01-01 00:28:21,5.66,90,74


### 特徵整理,統一命名
HVFHV:'hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time'
Yellow:'tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID'


    newname (HVFHV/Yellow/Green)
    1.TLC 許可證號 : Name(hvfhs_license_num/--/--)
        *改為名稱標記 HV0003-uber/HV0005-lyft/yellow
    2.上車時間 : BoardingTime (pickup_datetime/tpep_pickup_datetime/lpep_pickup_datetime)
    3.下車時間 : DisembarkingTime (dropoff_datetime/tpep_dropoff_datetime/lpep_dropoff_datetime)
    4.上車地點ID : PULocationID (PULocationID/PULocationID/PULocationID)
    5.下車地點ID : DOLocationID (DOLocationID/DOLocationID/DOLocationID)
    6.行駛里程 : TripMiles (trip_miles/trip_distance/trip_distance)
    7.行駛時間(秒) : TripTime (trip_time/--/--)
        *驗證是否正確?
        *下車時間-上車時間
    8.共乘 : shared_match_flag (shared_match_flag/--/--)
    9.節日 : is_holiday(--/--/--)
    10.星期 : weekday(--/--/--)

In [6]:
# 記錄開始時間
start_time = time.time()

df_TLC = pd.DataFrame()
#名稱統一
rename_dict_fhvhv = {'hvfhs_license_num':'Name','pickup_datetime': 'BoardingTime', 'dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_miles': 'TripMiles', 'trip_time': 'TripTime', 'base_passenger_fare': 'BaseFare','shared_match_flag':'shared_match_flag'}
df_fhvhv.rename(columns=rename_dict_fhvhv, inplace=True)
rename_dict_yellow = {'tpep_pickup_datetime': 'BoardingTime', 'tpep_dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_distance': 'TripMiles', 'fare_amount': 'BaseFare'}
df_yellow.rename(columns=rename_dict_yellow, inplace=True)

#fhvhv_Name
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0003', 'Name'] = 'uber'
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0005', 'Name'] = 'lyft'

#yellow 新增特徵 TripTime,Name
df_yellow['TripTime'] = (df_yellow['DisembarkingTime'] - df_yellow['BoardingTime']).dt.total_seconds()
df_yellow['Name'] = 'yellow'
df_yellow['shared_match_flag'] = 'N'
#yellow合併到fhvhv
df_yellow = df_yellow.reindex(columns = df_fhvhv.columns)
df_TLC = pd.concat([df_fhvhv, df_yellow], ignore_index=True)


# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")

#清理暫存
del df_yellow ,df_fhvhv
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 132.76 秒
done時間: 2023-08-17 16:39:52.327836


### 紀錄初始資料筆數

In [7]:
#紀錄初始資料筆數
init_df_TLC_shape = df_TLC.shape
init_df_uber_shape = df_TLC[df_TLC['Name']=='uber'].shape
init_df_lyft_shape = df_TLC[df_TLC['Name']=='lyft'].shape
init_df_yellow_shape = df_TLC[df_TLC['Name']=='yellow'].shape
print('Name=init_data_row=init_data_column')
print(f'TLC={init_df_TLC_shape[0]}={init_df_TLC_shape[1]}')
print(f'uber={init_df_uber_shape[0]}={init_df_uber_shape[1]}')
print(f'lyft={init_df_lyft_shape[0]}={init_df_lyft_shape[1]}')
print(f'yellow={init_df_yellow_shape[0]}={init_df_yellow_shape[1]}')

print("\ndone時間:", datetime.datetime.now())

Name=init_data_row=init_data_column
TLC=311527213=8
uber=164845019=8
lyft=53275729=8
yellow=76896949=8

done時間: 2023-08-17 16:43:27.259361


### 缺失值確認

In [8]:
print(df_TLC.isnull().sum())

Name                 0
BoardingTime         0
DisembarkingTime     0
PULocationID         0
DOLocationID         0
TripMiles            0
TripTime             0
shared_match_flag    0
dtype: int64


### 移除非目標車商

In [9]:
#查看目前有的車商
print(df_TLC['Name'].unique())

print("\ndone時間:", datetime.datetime.now())

['uber' 'lyft' 'HV0002' 'HV0004' 'yellow']

done時間: 2023-08-17 16:45:38.635451


In [10]:
#筆數計算-1
print('HV0002=筆數=占比')
print('\=', df_TLC[(df_TLC['Name'] == 'HV0002')].shape[0],'=',round(df_TLC[(df_TLC['Name'] == 'HV0002')].shape[0] / init_df_TLC_shape[0]*100,4),'%')

print("\ndone時間:", datetime.datetime.now())

HV0002=筆數=占比
\= 6388934 = 2.0508 %

done時間: 2023-08-17 16:47:21.387790


In [11]:
#筆數計算-2
print('HV0004=筆數=占比')
print('\=', df_TLC[(df_TLC['Name'] == 'HV0004')].shape[0],'=',round(df_TLC[(df_TLC['Name'] == 'HV0004')].shape[0] / init_df_TLC_shape[0]*100,4),'%')

print("\ndone時間:", datetime.datetime.now())

HV0004=筆數=占比
\= 10120582 = 3.2487 %

done時間: 2023-08-17 16:47:49.705255


In [12]:
#移除
df_TLC = df_TLC.drop(df_TLC[df_TLC['Name'] == 'HV0002'].index)
df_TLC = df_TLC.drop(df_TLC[df_TLC['Name'] == 'HV0004'].index)
print(df_TLC['Name'].unique())

print("\ndone時間:",datetime.datetime.now())

['uber' 'lyft' 'yellow']

done時間: 2023-08-17 16:54:29.329573


### 移除上車地點 PULocationID 未知 (264,265)

In [13]:
#筆數計算
print('264筆數=265筆數=總占比') 
print(df_TLC[(df_TLC['PULocationID'] == 264)].shape[0],'=', df_TLC[(df_TLC['PULocationID'] == 265)].shape[0],'=', 
       round(df_TLC[(df_TLC['PULocationID'] == 264) | (df_TLC['PULocationID'] == 265)].shape[0] / init_df_TLC_shape[0]*100,4),'%'  )

print("\ndone時間:", datetime.datetime.now())

264筆數=265筆數=總占比
627327 = 53304 = 0.2185 %

done時間: 2023-08-17 16:56:06.882191


In [14]:
df_TLC = df_TLC[~df_TLC['PULocationID'].isin([264, 265])]

#check-必須為0
print('上車未知剩餘筆數 = ',df_TLC[(df_TLC['PULocationID'] == 264) | (df_TLC['PULocationID'] == 265)].shape[0])
print("\ndone時間:", datetime.datetime.now())

上車未知剩餘筆數 =  0

done時間: 2023-08-17 16:58:15.413238


### 移除不符合的共乘資料
    UBER 2020/3/17-2022/6/20
    LYFT 2020/3/17-2021/8/18
         2023/5/12-

In [15]:
#uber,lyft
# 定義起始日期和結束日期
uber_start_date = pd.to_datetime('2020-03-17')
uber_end_date = pd.to_datetime('2022-06-20')
lyft_start_date = pd.to_datetime('2020-03-17')
lyft_end_date = pd.to_datetime('2021-08-18')
lyft_start_date1 = pd.to_datetime('2023-05-12')

#筆數計算
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date))).sum()

temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 ))).sum()
print('/=uber=lyft')
print('data=',temp_uber,'=',temp_lyft)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,4),'%','=',round(temp_uber / init_df_uber_shape[0]*100,4),'%' )

print("\ndone時間:", datetime.datetime.now())

#清理暫存
del temp_uber ,temp_lyft

/=uber=lyft
data= 0 = 0
占比= 0.0 % = 0.0 %

done時間: 2023-08-17 17:05:22.799563


###　不處理  trip_time != DisembarkingTime - BoardingTime
### 移除 TripTime <= 0

In [16]:
#記錄開始時間
start_time = time.time()

#印出資訊
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['TripTime'] <= 0)).sum()
temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['TripTime'] <= 0)).sum()
temp_yellow = ((df_TLC['Name'] == 'yellow') & (df_TLC['TripTime'] <= 0)).sum()

print('\=uber=lyft=yellow')
print('data=',temp_uber,'=',temp_lyft,'=',temp_yellow)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,5),'%='
              ,round(temp_lyft / init_df_lyft_shape[0]*100,5),'%='
              ,round(temp_yellow / init_df_yellow_shape[0]*100,5),'%' )

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

\=uber=lyft=yellow
data= 2033 = 11738 = 63687
占比= 0.00123 %= 0.02203 %= 0.08282 %

程式執行花費的時間： 33.43 秒
done時間: 2023-08-17 17:06:21.970749


In [17]:
#移除 (df_TLC['TripTime'] <= 0)
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripTime'] <= 0].index)
#check-必須為0
print('TripTime<=0剩餘筆數:',(df_TLC['TripTime'] <= 0).sum())
print("\ndone時間:", datetime.datetime.now())

TripTime<=0剩餘筆數: 0

done時間: 2023-08-17 17:10:13.131483


### 移除TripMiles <= 0

In [18]:
#記錄開始時間
start_time = time.time()

#印出資訊
temp_uber = ((df_TLC['Name'] == 'uber') & (df_TLC['TripMiles'] <= 0)).sum()
temp_lyft = ((df_TLC['Name'] == 'lyft') & (df_TLC['TripMiles'] <= 0)).sum()
temp_yellow = ((df_TLC['Name'] == 'yellow') & (df_TLC['TripMiles'] <= 0)).sum()

print('\=uber=lyft=yellow')
print('data=',temp_uber,'=',temp_lyft,'=',temp_yellow)
print('占比=',round(temp_uber / init_df_uber_shape[0]*100,5),'%='
              ,round(temp_lyft / init_df_lyft_shape[0]*100,5),'%='
              ,round(temp_yellow / init_df_yellow_shape[0]*100,5),'%' )


#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")

del temp_uber,temp_lyft,temp_yellow
print("done時間:",datetime.datetime.now())

\=uber=lyft=yellow
data= 68957 = 405109 = 551949
占比= 0.04183 %= 0.7604 %= 0.71778 %

程式執行花費的時間： 36.51 秒
done時間: 2023-08-17 17:11:23.614158


In [19]:
#移除 (df_TLC['TripMiles'] <= 0)
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripMiles'] <= 0].index)
#check-必須為0
print('TripMiles<=0剩餘筆數:',(df_TLC['TripMiles'] <= 0).sum())
print("\ndone時間:", datetime.datetime.now())

TripMiles<=0剩餘筆數: 0

done時間: 2023-08-17 17:15:28.260947


### 新增節日相關資訊

In [20]:
# from holidays.calendars import gregorian
class AllHolidays(US):
    def _populate(self, year):
        super()._populate(year)
        self.pop_named("Lincoln's Birthday")
        self.pop_named("Susan B. Anthony Day")
        
        self._add_holiday("Valentine's Day", 2, 14)
        self._add_holiday("St.Patrick's Day", 3, 17)
        self._add_holiday("Halloween", 10, 31)
        
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=4))
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=5))
#        self._add_holiday("Thanksgiving weekend", 
#                          self._get_nth_weekday_of_month(month=11, n=4, weekday=6))
        
        self._add_holiday("Christmas Eve", 12, 24)
        self._add_holiday("Christmas season", 12, 26)
        self._add_holiday("Christmas season", 12, 27)
        self._add_holiday("Christmas season", 12, 28)
        self._add_holiday("Christmas season", 12, 29)
        self._add_holiday("Christmas season", 12, 30)
        
        self._add_holiday("New Year's Eve", 12, 31)

# 實例化AllHolidays
ny_holidays_all = AllHolidays(subdiv='NY', years=[2019, 2020, 2021, 2022, 2023])

# 新增2019~2023年的復活節、復活節星期一
ny_holidays_all[datetime.date(2019, 4, 21)] = "Easter"
ny_holidays_all[datetime.date(2019, 4, 22)] = "Easter Monday"
ny_holidays_all[datetime.date(2020, 4, 12)] = "Easter"
ny_holidays_all[datetime.date(2020, 4, 13)] = "Easter Monday"
ny_holidays_all[datetime.date(2021, 4, 4)] = "Easter"
ny_holidays_all[datetime.date(2021, 4, 5)] = "Easter Monday"
ny_holidays_all[datetime.date(2022, 4, 17)] = "Easter"
ny_holidays_all[datetime.date(2022, 4, 18)] = "Easter Monday"
ny_holidays_all[datetime.date(2023, 4, 9)] = "Easter"
ny_holidays_all[datetime.date(2023, 4, 10)] = "Easter Monday"

# 新增2019~2023感恩節
ny_holidays_all[datetime.date(2019,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 29)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 30)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 26)] =  "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 23)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 25)] = "Thanksgiving weekend"

# 查看
for i in sorted(ny_holidays_all.items()):
    print(i)

del i

(datetime.date(2019, 1, 1), "New Year's Day")
(datetime.date(2019, 1, 21), 'Martin Luther King Jr. Day')
(datetime.date(2019, 2, 14), "Valentine's Day")
(datetime.date(2019, 2, 18), "Washington's Birthday")
(datetime.date(2019, 3, 17), "St.Patrick's Day")
(datetime.date(2019, 4, 21), 'Easter')
(datetime.date(2019, 4, 22), 'Easter Monday')
(datetime.date(2019, 5, 27), 'Memorial Day')
(datetime.date(2019, 7, 4), 'Independence Day')
(datetime.date(2019, 9, 2), 'Labor Day')
(datetime.date(2019, 10, 14), 'Columbus Day')
(datetime.date(2019, 10, 31), 'Halloween')
(datetime.date(2019, 11, 5), 'Election Day')
(datetime.date(2019, 11, 11), 'Veterans Day')
(datetime.date(2019, 11, 28), 'Thanksgiving; Thanksgiving weekend')
(datetime.date(2019, 11, 29), 'Thanksgiving weekend')
(datetime.date(2019, 11, 30), 'Thanksgiving weekend')
(datetime.date(2019, 12, 24), 'Christmas Eve')
(datetime.date(2019, 12, 25), 'Christmas Day')
(datetime.date(2019, 12, 26), 'Christmas season')
(datetime.date(2019, 12, 

In [22]:
# 增加所有節日標籤
# all_data為Uber, Lyft, Yellow合併後的DataFrame，再請自行修改名稱
df_TLC['is_holiday'] = np.where(df_TLC['BoardingTime'].dt.date.isin(ny_holidays_all), True, False)
print(df_TLC['is_holiday'].value_counts())

print('-------------')
print('Name=True=False')
print('uber=',((df_TLC['Name'] == 'uber') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'uber') & (df_TLC['is_holiday'] == False)).sum())
print('lyft=',((df_TLC['Name'] == 'lyft') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'lyft') & (df_TLC['is_holiday'] == False)).sum())
print('yellow=',((df_TLC['Name'] == 'yellow') & (df_TLC['is_holiday'] == True)).sum(),'=',((df_TLC['Name'] == 'yellow') & (df_TLC['is_holiday'] == False)).sum())
print('Total=',(df_TLC['is_holiday'] == True).sum(),'=',(df_TLC['is_holiday'] == False).sum())

print("\ndone時間:", datetime.datetime.now())



is_holiday
False    275858796
True      17374797
Name: count, dtype: int64
-------------
Name=True=False
uber= 10096481 = 154671349
lyft= 3253226 = 49601627
yellow= 4025090 = 71585820
Total= 17374797 = 275858796

done時間: 2023-08-17 13:51:43.736654


### 新增星期特徵
    #星期幾 (用上車時間 BoardingTime)
    #(一,二,三,四,五,六,日) = (0,1,2,3,4,5,6)

In [21]:
#記錄開始時間
start_time = time.time()

df_TLC['weekday'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.weekday
df_TLC['date'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.date
df_TLC['year'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.year
df_TLC['month'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.month
df_TLC['day'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.day
df_TLC['hour'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.hour

#記錄結束時間
end_time = time.time()
#計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 144.01 秒
done時間: 2023-08-17 17:18:03.429156
